# London Crime Analysis using SQL Big Query
SQL = Structure Query Language, is the programming language used with databases.
BigQuery = web service that lets you appl SQL to huge datasets.

In this exercise, we learn the basic of accessing and examining BigQuery datasets.

Background story:
Bob is Brazil citizen who wants to move to London because of work. Because of high crime rates in Brazil, Bob trying to find more peaceful life in London. Before he moved to London, Bob do a research about crime in London so he could avoid to experience any sort of crime. There are several questions that must be answered by Bob after he do his research.

The purpose of exploring the dataset is to answer these following questions:
1. If a person wants to move to London in peaceful area, where it would be?
2. What crime should we be aware of in that area?
3. What month will be the most dangerous month to be more vigilant towards the crime?

# 1. Data Preparation
To use BigQuery, we need to import Python package below:

In [1]:
from google.cloud import bigquery
import plotly.graph_objs as go
import plotly.express as px
import json

# Create a "Client" object
crime = bigquery.Client()

dataset_ref = crime.dataset("london_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = crime.get_dataset(dataset_ref)

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("crime_by_lsoa")

# API request - fetch the table
table = crime.get_table(table_ref)

# Preview the first five lines of the table
crime.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000478,Brent,Sexual Offences,Rape,0,2008,1
1,E01000619,Brent,Sexual Offences,Rape,1,2008,1
2,E01000619,Brent,Sexual Offences,Rape,0,2008,7
3,E01000156,Barnet,Sexual Offences,Rape,0,2008,11
4,E01000156,Barnet,Sexual Offences,Rape,0,2008,7


# 2. Data Exploration and Visualisation

In [2]:
#Find out borough with most crime
borough_count = """
                SELECT borough, SUM(value) AS num_crimes
                FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
                GROUP BY borough ORDER BY num_crimes DESC
                """
borough_count = crime.query(borough_count).result().to_dataframe()
borough_count

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,borough,num_crimes
0,Westminster,455028
1,Lambeth,292178
2,Southwark,278809
3,Camden,275147
4,Newham,262024
5,Croydon,260294
6,Ealing,251562
7,Islington,230286
8,Tower Hamlets,228613
9,Brent,227551


In [3]:
fig = px.bar(borough_count, x= "num_crimes", y="borough", title = 'Borough in Greater Area of London with Most Crimes')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

In [4]:
#Find out most minor crime category that happened in Greater London Area
minor_crime_count1 = """
              SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
              GROUP BY minor_category,major_category ORDER BY num_crimes DESC
              """
minor_crime_count1 = crime.query(minor_crime_count1).result().to_dataframe()
minor_crime_count1

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft,Theft and Handling,980085
1,Theft From Motor Vehicle,Theft and Handling,569956
2,Burglary in a Dwelling,Burglary,491282
3,Harassment,Violence Against the Person,458124
4,Assault with Injury,Violence Against the Person,451001
5,Possession Of Drugs,Drugs,431948
6,Common Assault,Violence Against the Person,413690
7,Theft From Shops,Theft and Handling,345142
8,Other Theft Person,Theft and Handling,308842
9,Criminal Damage To Motor Vehicle,Criminal Damage,265463


In [5]:
fig = px.bar(minor_crime_count1, x= "num_crimes", y="minor_category", title = 'Minor Category of Crimes in Greater Area of London')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the graph and table above, we know that minor category of crimes that most happened in Greater London Area is Other Theft with total number of crime that occured is 980.085 and minor category of crimes that least happened in Greater London Area is Rape with total number of crimes that occured is 268.

In [6]:
#Find out most major crime category that happened in Greater London Area
major_crime_count1 = """
                    SELECT DISTINCT major_category, SUM(value) AS num_crimes
                    FROM `bigquery-public-data.london_crime.crime_by_lsoa` 
                    GROUP BY major_category ORDER BY num_crimes DESC
                    """
major_crime_count1 = crime.query(major_crime_count1).result().to_dataframe()
major_crime_count1

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,major_category,num_crimes
0,Theft and Handling,2661861
1,Violence Against the Person,1558081
2,Burglary,754293
3,Criminal Damage,630938
4,Drugs,470765
5,Robbery,258873
6,Other Notifiable Offences,106349
7,Fraud or Forgery,5325
8,Sexual Offences,1273


In [7]:
fig = px.bar(major_crime_count1, x= "num_crimes", y="major_category", title = 'Major Category of Crimes in Greater Area of London')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the graph and table above, we know that major category of crimes that most happened in Greater London Area is Theft and Handling with total number of crime that occured is 2.661.861 and major category of crimes that least happened in Greater London Area is Sexual Offences with total number of crimes that occured is 1273.

Now we know the major and minor crimes that most happened in Greater London Area and otherwise. Lets find out if the same type of crimes that most happened and otherwise is the same type of crimes that happened in City of London.

In [8]:
#Find out most major crime category that happened in City of London
minor_crime_count_london = """
              SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London'
              GROUP BY minor_category,major_category ORDER BY num_crimes DESC
              """
minor_crime_count_london = crime.query(minor_crime_count_london).result().to_dataframe()
minor_crime_count_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft,Theft and Handling,270
1,Other Theft Person,Theft and Handling,194
2,Theft/Taking of Pedal Cycle,Theft and Handling,41
3,Harassment,Violence Against the Person,36
4,Assault with Injury,Violence Against the Person,29
5,Possession Of Drugs,Drugs,28
6,Common Assault,Violence Against the Person,27
7,Personal Property,Robbery,23
8,Theft From Shops,Theft and Handling,22
9,Theft From Motor Vehicle,Theft and Handling,18


In [9]:
fig = px.bar(minor_crime_count_london, x= "num_crimes", y="minor_category", title = 'Minor Category of Crimes in City of London')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the graph and table above, we know that minor category of crimes that most happened in City of London is Other Theft with total number of crime that occured is 270 and minor category of crimes that least happened in Greater London Area is Robbery of Business Property with total number of crimes that occured is 1. It is also interesting to know that none of the murder crime that happened in City of London

In [10]:
#Find out most major crime category that happened in City of London
major_crime_count_london = """
              SELECT DISTINCT major_category, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London'
              GROUP BY major_category ORDER BY num_crimes DESC
              """
major_crime_count_london = crime.query(major_crime_count_london).result().to_dataframe()
major_crime_count_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,major_category,num_crimes
0,Theft and Handling,561
1,Violence Against the Person,114
2,Drugs,33
3,Robbery,24
4,Other Notifiable Offences,17
5,Criminal Damage,16
6,Burglary,15


In [11]:
fig = px.bar(major_crime_count_london, x= "num_crimes", y="major_category", title = 'Major Category of Crimes in City of London')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the graph and table above, we know that major category of crimes that most happened in City of London is Theft and Handling with total number of crimes that occured is 561 and major category of crimes that least happened in City of London is Burglary with total number of crimes that occured is 15. It is also interesting to know that none of sexual offence and fraud or forgery crimes that happened in the City of London

In [12]:
#Crime that Happened per Year in Greater London Area
crime_year  = """
              SELECT year, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              GROUP by year
              ORDER by year
              """
crime_year  = crime.query(crime_year).result().to_dataframe()
crime_year

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,year,num_crimes
0,2008,738641
1,2009,717214
2,2010,715324
3,2011,724915
4,2012,737329
5,2013,686407
6,2014,680183
7,2015,711624
8,2016,736121


In [13]:
px.line(crime_year,x="year",y="num_crimes",title='Number of Crimes in Greater London Area per Year')

The chart above shows that since 2014, the number of crimes has been increasing. While the year with highest crimes is year 2008 with number of crimes that occured is 738.641, year 2012 with number of crimes that occured is 737.329, and year 2016 with number of crimes that occured is 736.121

Now lets see the number of crimes that happened per month in year 2016

In [14]:
#Crime that Happened per Months (2016) in Greater London Area
crime_month = """
              SELECT month, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE year = 2016
              GROUP by month
              ORDER by month
              """
crime_month = crime.query(crime_month).result().to_dataframe()
crime_month

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,month,num_crimes
0,1,58847
1,2,56697
2,3,59167
3,4,58637
4,5,63990
5,6,62262
6,7,65519
7,8,62666
8,9,61412
9,10,63405


In [15]:
px.line(crime_month,x="month",y="num_crimes",title='Number of Crimes in Greater London Area per Month (2016)')

The chart above shows that the number of crimes is raise significant from April to May with the number of crimes that happened is 58.637 crimes to 63.990 crimes. While May is month with biggest the increase of crimes, the month with highest number of crimes is July with 65.519 crimes. Now lets see the category of crimes that happened in July 2016

In [16]:
#Minor Crime in Greater London Area on July 2016
minor_crime_july_2016 = """
                        SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
                        FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                        WHERE year = 2016 AND month = 7
                        GROUP BY minor_category,major_category ORDER BY num_crimes DESC
                        """
minor_crime_july_2016 = crime.query(minor_crime_july_2016).result().to_dataframe()
minor_crime_july_2016

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft,Theft and Handling,9291
1,Harassment,Violence Against the Person,7738
2,Common Assault,Violence Against the Person,5699
3,Assault with Injury,Violence Against the Person,4728
4,Theft From Motor Vehicle,Theft and Handling,4248
5,Theft From Shops,Theft and Handling,3963
6,Burglary in a Dwelling,Burglary,3239
7,Other Theft Person,Theft and Handling,3053
8,Possession Of Drugs,Drugs,2995
9,Theft/Taking Of Motor Vehicle,Theft and Handling,2337


In [17]:
fig = px.bar(minor_crime_july_2016, x= "num_crimes", y="minor_category", title = 'Minor Category of Crimes in Greater Area of London on July 2016')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the graph and table above, we know that minor category of crimes that most happened in Greater London Area on July 2016 is Other Theft with total number of crimes that occured is 9291. It is same minor category of crimes that most happened in Greater London Area from 2008 - 2014. Thankfully, there is none of the Rape, Other Sexual, Counted per Victim, and Other Fraud & Forgery that happened.

In [18]:
#Crime that Happened per Year in City of London
crime_year_london  = """
              SELECT year, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London'
              GROUP by year
              ORDER by year
              """
crime_year_london  = crime.query(crime_year_london).result().to_dataframe()
crime_year_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,year,num_crimes
0,2008,0
1,2009,0
2,2010,0
3,2011,69
4,2012,157
5,2013,110
6,2014,115
7,2015,151
8,2016,178


In [19]:
px.line(crime_year_london,x="year",y="num_crimes",title='Number of Crimes in City of London per Year')

The chart above shows that there are huge increase of the crimes from 2010 - 2012 with the number of crimes is increased from 0 - 157. After the number of crimes decrease from 2012 - 2013, the number of crimes is increase from 2014 - 2016. The highest number of crimes is on 2016 with the number of crimes that happened is 178. Now lets see the number of crimes that happened per month in year 2016

In [20]:
#Crime that Happened per Months in City of London
crime_month_london = """
              SELECT month, SUM(value) AS num_crimes
              FROM `bigquery-public-data.london_crime.crime_by_lsoa`
              WHERE borough = 'City of London' AND
                    year = 2016
              GROUP by month
              ORDER by month
              """
crime_month_london = crime.query(crime_month_london).result().to_dataframe()
crime_month_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,month,num_crimes
0,1,10
1,2,10
2,3,12
3,4,15
4,5,9
5,6,8
6,7,13
7,8,19
8,9,19
9,10,17


In [21]:
px.line(crime_month_london,x="month",y="num_crimes",title='Number of Crimes in City of London per Month (2016)')

While the number of crime is raise significantly from April to May in Greater London Area, its different in City of London with the number of crimes is raise significantly from November to December with the number of crimes is increased from 12 to 34 crimes. The highest number of crimes is happened on December with the number of crimes is 34. Lets find out the category of crimes that happened on December 2016 in the City of London

In [22]:
#minor crime in City of London on December 2016
minor_crime_december_2016_london = """
                               SELECT DISTINCT minor_category,major_category, SUM(value) AS num_crimes
                               FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                               WHERE borough = 'City of London'AND year = 2016 AND month = 7
                               GROUP BY minor_category,major_category ORDER BY num_crimes DESC
                               """
minor_crime_december_2016_london = crime.query(minor_crime_december_2016_london).result().to_dataframe()
minor_crime_december_2016_london

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,major_category,num_crimes
0,Other Theft Person,Theft and Handling,3
1,Other Theft,Theft and Handling,3
2,Harassment,Violence Against the Person,2
3,Wounding/GBH,Violence Against the Person,1
4,Assault with Injury,Violence Against the Person,1
5,Possession Of Drugs,Drugs,1
6,Common Assault,Violence Against the Person,1
7,Criminal Damage To Other Building,Criminal Damage,1
8,Theft/Taking Of Motor Vehicle,Theft and Handling,0
9,Other violence,Violence Against the Person,0


In [23]:
fig = px.bar(minor_crime_december_2016_london, x= "num_crimes", y="minor_category", title = 'Minor Category of Crimes in City of London on December 2016')
fig.update_layout(yaxis={'categoryorder':'total ascending'})

From the table above, we know that minor category of crimes that most happened in City of London on December 2016 is Other Theft with total number of crimes that occured is 3. It is same minor category of crimes that most happened in City of London from 2008 - 2014.

# 3. Conclusion
If Bob wants to have more peaceful life, he should move to the City of London in Greater Area of London because of the lowest number of crime that happened from 2008 - 2016. Even City of London is most peaceful borough in Greater London Area, Bob must still careful because of the increase number of crime from 2014 - 2016. Bob should be aware of the crimes like theft and assault, especially on December.